In [1]:
from collections.abc import Generator

In [2]:
def averager() -> Generator[float, float, None]:
    total = .0
    count = .0
    average = .0
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

coro_avg = averager()

<span style="font-size: 15px;
             font-family: Courier New;">
    The first call of next makes the coroutine advance to the yield,
    <br>which is also known as 'priming the coroutine'
    <br>also you can do (or would have to) send None at first 
</span>

In [3]:
#coro_avg.send(None)
next(coro_avg)

0.0

In [4]:
coro_avg.send(10)

10.0

<span style="font-size: 15px;
             font-family: Courier New;">
    After each activation the coroutine is suspended precisely at the yield keyword, 
    <br> waiting for a value to be send.
    <br> the yield expression resovles to the value 10,
    <br> assigning it to the term variable
    <br> the rest of the loop updates the variables
</span>

In [5]:
coro_avg.send(30)

20.0

In [6]:
coro_avg.send(5)

15.0

In [7]:
#explicit termination (optional)
coro_avg.close()

-------

### Returning a Value from a Coroutine

In [8]:
from typing import Union, NamedTuple
from typing import TypeAlias

In [9]:
class Result(NamedTuple):
    count: int # type: ignore
    average: float
        
class Sentinel:
    def __repr__(self):
        return f'<Sentinel>'
    
STOP = Sentinel()

In [10]:
#SentType = Union[float, Sentinel]
#SendType: TypeAlias = float | Sentinel

In [11]:
def averager2(verbose: bool = False) -> Generator[None, float | Sentinel, Result]:
    total = .0
    count = .0
    average = .0
    while True:
        term = yield #this yields None but recieves a term from .send()
        if verbose:
            print('received:', term)
        if isinstance(term, Sentinel):
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)    

In [12]:
coro_avg = averager2()
next(coro_avg)

In [13]:
coro_avg.send(10)

In [14]:
coro_avg.send(30)
coro_avg.send(6.5)

In [15]:
coro_avg.close()
#The GeneratorExit exception is raised at the yield line in the coroutine,
#so the return statement is never reached

In [16]:
coro_avg = averager2()
next(coro_avg)

coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)

try: 
    coro_avg.send(STOP)
except StopIteration as exc:
    result = exc.value #wtf?
    
result 

Result(count=3.0, average=15.5)

In [17]:
def compute(f, *a):
    res = yield from f(*a)
    print('computed:', res)
    return res

comp = compute(averager2, [True])

In [18]:
for v in [None, 10, 20, 30, STOP]:
    try:
        comp.send(v)
    except StopIteration as exc:
        result = exc.value
        
result

received: 10
received: 20
received: 30
received: <Sentinel>
computed: Result(count=3.0, average=20.0)


Result(count=3.0, average=20.0)

In [28]:
def generator():
    a = 0
    while True:
        b = yield
        a += b
        if isinstance(b, float): 
            break
    return a

#you should use isinstance to raise StopIteration if you want to use coroutines like that.
#for it can fuck up the result variable

In [20]:
my_gen = generator()

In [21]:
next(my_gen)

In [22]:
my_gen.send(9)

In [23]:
my_gen.send(8)

In [24]:
try:
    my_gen.send(11.0)
except StopIteration as exc:
    result = exc.value
    
result

28.0

In [29]:
my_comp = compute(generator)
for v in [None, 10, 20, 30.0]:
    try:
        my_comp.send(v)
    except StopIteration as exc:
        result = exc.value
        
result

computed: 60.0


60.0

In [26]:
sum(list(range(12))) #does it make sense?

66

----------------